In [24]:
%matplotlib inline

In [25]:
# Author: Yulou Zhou
#Weight loss function by commonality (and normalize)
#padded sequence
#for loop to test parameters
#for loop to get prev generated chord

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from MyDataset import MyDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
torch.manual_seed(1)

In [26]:
from torch.utils.data import dataloader
def collate_fn(batch):
    batch.sort(key=lambda x: x['mask'], reverse=True)
    #for i in batch:
        #print(i['mask'])
    return dataloader.default_collate(batch)

In [27]:
batch_size = 256
train_data = MyDataset('relativedata', prevchord = 4)
weights = train_data.stats()

In [28]:
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn = collate_fn)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    print("marr: ", i['marr'])
    print("l: ", i['l'])
    c += 10
    if c > 0: break

p:  tensor([[  0,   0,   0,  ...,   3,   4,   7],
        [ 10,   4,   7,  ...,   8,   4,   7],
        [  6,   4,   7,  ...,   6,   4,   7],
        ...,
        [  6,   3,   6,  ...,   7,   4,   7],
        [  0,   0,   0,  ...,   0,   0,   0],
        [  0,   0,   0,  ...,   0,   0,   0]])
m:  tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  8,   3,   3,  ...,   0,   0,   0],
        [ 10,  10,  10,  ...,   0,   0,   0],
        ...,
        [ 11,  11,   1,  ...,   0,   0,   0],
        [  0,  10,  11,  ...,   0,   0,   0],
        [  1,   3,   6,  ...,   0,   0,   0]])
marr:  tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], dtype=torch.float64)
l:  tensor([[  3,   4,   7],
        [ 12,   3,   7],
        [  8,   4,   7],
        [  1,  

In [29]:
def normalize(array):
    return [i / sum(array) for i in array]
newweights = []

def weightloss(weights, eps):
    for w in weights:
        newweights.append(torch.FloatTensor( normalize([1/(i + eps) for i in w] ) ) )
    #print(newweights)


In [30]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        self.p_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        self.m_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim, bidirectional = False)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim, bidirectional = False)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear((p_hidden_dim + m_hidden_dim) , tagset_size)
        self.root2second = nn.Linear((p_hidden_dim + m_hidden_dim) + 1, tagset_size)
        self.root2third = nn.Linear((p_hidden_dim + m_hidden_dim) + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_one_hot(self, vocab_size):
        #initialize each embedding
        #stack them together (or other ways to have  pretrained embeddings)
        #pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #we should turn data into torch.from_numpy(pretrained_weight)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
        torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
    
    def init_hidden(self, hidden_dim):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, self.batch_size, hidden_dim),
                torch.zeros(1, self.batch_size, hidden_dim))


    def forward(self, prev, melody, mask):
        
        prev_embeds = self.p_embed(prev)
        x1 = torch.transpose(prev_embeds, 0, 1) 
        
        #TODO: ensure that melody input is a tensor
        
        melody_embeds =  self.m_embed(melody) #Error here
        x2 = torch.transpose(melody_embeds, 0, 1)
        packed = pack_padded_sequence(x2, mask, batch_first=False)
        
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(packed, self.m_hidden)
        (h_t, c_t) = self.m_hidden
        out_unpacked, _ = pad_packed_sequence(m_lstm_out, batch_first=False)
        #print(out_unpacked[-1], h_t[0])
        p_fstate, m_fstate = p_lstm_out[-1], h_t[0]

        
        concat = torch.cat((p_fstate, m_fstate), 1)
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        

        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        
        #stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)

        return [tag_scores, second_scores, third_scores]

In [33]:
import random
for iteration in range(1):

    P_HIDDEN_DIM = max(int(np.random.normal(30, 20)), 5)
    M_HIDDEN_DIM = max(int(np.random.normal(30, 20)), 5)
    vocab_size = 13
    tagset_size = 13
    EPS = random.uniform(0.005, 0.15)
    ITER = int(random.uniform(300, 1200))
    P_EMBEDDING_DIM = vocab_size
    M_EMBEDDING_DIM = vocab_size
    
    P_HIDDEN_DIM = 5
    M_HIDDEN_DIM = 5
    
    
    ITER = 12000

    print("iter: ", ITER, "  phid: ", P_HIDDEN_DIM, "  mhid: ", M_HIDDEN_DIM, "  eps: ", EPS)
    newweishts = weightloss(weights, EPS)
    model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
    loss_function = nn.NLLLoss(weight = newweights[0])
    loss_function2 = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.2)
    
    for epoch in range(ITER):  # again, normally you would NOT do 300 epochs, it is toy data
        for i in data_loader:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
            model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Tensors of word indices.
            p, m, l, mask, marr = i['p'], i['m'], i['l'], i['mask'], i['marr']
            #print(m)
            tag_scores = model(p, m, mask)
            # Step 3. Run our forward pass.

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            #print(targets[:1])
            loss = 0

            loss += loss_function(tag_scores[0], torch.t(l)[0])  
            loss += 0.25 * loss_function2(tag_scores[1], torch.t(l)[1])
            loss += 0.125 * loss_function2(tag_scores[2], torch.t(l)[2])   

            loss.backward()
            optimizer.step()
            break
        if(epoch % 100==0): 
            print(epoch, loss)
            accuracy = 0.0
            c = 0
            for j in range(2):
                for i in data_loader:
                    c+=1
                    p, m, l, mask, marr= i['p'], i['m'], i['l'], i['mask'], i['marr']
                    tag_scores = model(p, m, mask)
                    pred = tag_scores[0].max(1)[1].numpy()
                    truth = torch.t(l)[0].numpy()
                    print(pred[:20], truth[:20])
                    accuracy += np.average(pred == truth)
                    if (c>=1): break
            print("accuracy:", accuracy/c)



#     for j in range(1):
#         for i in data_loader:
#             c+=1
#             p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
#             tag_scores = model(p, m, mask)
#             pred = tag_scores[0].max(1)[1].numpy()
#             truth = torch.t(l)[0].numpy()
#             accuracy += np.average(pred == truth)
#             if (c>=1): break
#     print("accuracy:", accuracy/c)

iter:  12000   phid:  5   mhid:  5   eps:  0.06455417238716951
0 tensor(3.9526)
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8] [ 3  8  1  1 10  1  6  1  6  1 12  3 10  3  5  7  4  8  8  7]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8] [ 1  4 11  4  8  2  4  5  6  7  4 11 11  9  1  3  6  6  3 10]
accuracy: 0.17578125
100 tensor(2.8315)
[1 1 1 1 1 1 8 1 3 1 8 8 3 3 8 1 1 8 8 8] [10  1  1  3  1  2  5 11  8  6  5 11  3  6  8  1  6  4  7  6]
[1 3 8 8 3 8 8 3 8 1 8 8 1 8 1 8 8 8 3 3] [ 1 12  8  6  5  1  3  1  4  2  1 11  7  1  1 11  7  1  8  8]
accuracy: 0.171875
200 tensor(2.7498)
[8 8 1 1 8 4 8 1 4 4 8 1 4 1 8 1 1 8 8 1] [11 10  8  1 10 10 10 10  3  2  6  6  8  5 12  8  7  8 11  5]
[8 1 8 8 1 8 8 1 4 1 4 8 1 8 8 1 1 1 8 1] [ 7  3  1  4 11  6  8 11 11  3  4 11  2 11  6  3  3  8  1 10]
accuracy: 0.173828125
300 tensor(2.5503)
[ 3  3  1  3  1  1  8  1 11  4  8  3  4  1  3  3  4  8  8  1] [ 1  1 10  1  6  1 11  5  1  6 11  6  5  1 10 10 11  6  3  1]
[ 1 11  3  4  3  8  3  4  1 11  1  1  3  3  1  3  3  

2900 tensor(2.2298)
[ 8  3 10  6  9  1  9  8  5  4  6 11  5  8  3  8 10  6  5  5] [ 1 10 10  3  8  8  7  6  5  1  6 11  8  8  3  3  2  3 11  3]
[ 3  1  3  8 11  3  8  4  5  3 11  1 10 10  8 12  9 10  1  9] [ 6  4  8  8 11  1  3  4 10  8 11  1  3  3 11  9  1  3  1  4]
accuracy: 0.26171875
3000 tensor(2.0991)
[ 5  5  1 10  5  3  6  4  1  1  3  8 11  3  3  3 11  6 10  2] [ 5  9  3 10 10  3  1 10 12  1  8  1  4  3  8 10 11  1  3  4]
[10  3  3 10  3  1  1  6  8  8  1  6  6  8  1 10  1  2  8  6] [ 8  8  3 10  5 10  3 11  8  8 11  6  6 10  1  7  6 11  3  6]
accuracy: 0.298828125
3100 tensor(2.1552)
[ 6  3  3  3  3  5  5  3  5  1  3  8  9 10  7  3  8  1  8  5] [ 9  3 10  3  3 10  5  8  5 10 10  4  4 12 12 10  1  6  9 10]
[ 1  6 10  5  1  1  3  8  5  3  5  1 11 11  6  2  6  9  5  3] [ 1  8  3 10 10  1  3  1  3  3 10  4 11  2  1  3  3  4  8  3]
accuracy: 0.275390625
3200 tensor(2.1989)
[ 6 10 10  8  8  5 10  9  8  8  6 10  2 10  1  5  5 11  1 12] [ 1  5  5 12  8  9 10  7 10  8  8  5  5  6 12  9 

5800 tensor(2.0472)
[ 3  2  8 10  5  4  1 10  1  6  1  5  4  5  8  4  1 11 10 10] [ 1  5  8 10  5  4  1  7  1  8  1  8  3  5  9 11  3 11  3  1]
[10 12  8 12  5  1  8  6 10 11  4  8  6  5  9 10  9  8  5  9] [ 4  5  8  2 12  6  8  6  7  6  1  8  8  2  9  5  6  3  5  4]
accuracy: 0.28515625
5900 tensor(2.1158)
[ 1 10 12 10  1 11  5  4 10 11  4 10 11 12  8 10  8  1 10  0] [ 8  8 12  7  1 11  8  4 10  6  4  8  4  5  6  1  1  1  1  3]
[ 8  5  3  8  3 10 10 11  1  4  6  4 11  3  3  8 10 10  1  8] [ 3 12  8  6 11  9  6  6 10  7  5  9 11  3  8  6  5  8  1  8]
accuracy: 0.298828125
6000 tensor(2.2527)
[ 8  5  6  2  8 10  3  8  8  9 11  1  5  9 11  5 10  8  3 10] [ 8  5  8  9  6  5  8  8  8  4  4  8 10  7 11  3  3 12  6 10]
[ 8  5  3  4  6  5  1  5  3  8 11 10 10  6  6  4  5  1  5  3] [ 8  8  3  9  6 11  8  7 10  1  4  8  5  6  6 11  1  6 10  3]
accuracy: 0.326171875
6100 tensor(2.2319)
[ 5 10  8  6  8  8 12  8  9  3 10  3  3  8  1  5  9  1  3 11] [ 9 10  3  1  3  1  1  3 11  3  6  8  6  1  6  8 

8700 tensor(2.1482)
[ 2  9  4  4  1  8 10  1  2 11 11  5  9 11  3  1  3  5  6  1] [ 5  6  1  8  1  4 10  6  0  6  6 11  8 11 11  6  1  2  6  1]
[ 5  2  4  4 10  5  3  4  6  5  1  5  8  1  8  8  3  2  6  8] [ 4  6  8  6 10 10  8  4  1  5  1 10  8  6  1  7  5  2  5  6]
accuracy: 0.2734375
8800 tensor(2.0941)
[ 3  1 10  3 10  3  6 11  6  9  4  1  6 11  8 12 10  1  1 10] [10 11  3  1  5  3  6  9  1  8  9  4  1 11 12  2 10  1  3 10]
[12  1  1  1  8  8  3  8  6  6  4  9 10 11 10  2  8  8  3  8] [ 5  1  3  6  3 10  7  8 10  6  4  8  5  6 10  4  6  8  8 10]
accuracy: 0.291015625
8900 tensor(2.1121)
[ 5 11  0  4  9  5  1  4  1  3 12 10  8  1  4  8  1 10  6 12] [10  6  1  4  4  5 11  8  8  3 10 10  1  3  4  1 11  1  6  7]
[ 5  3  6 10  2  5  1  6 10  5  9  5 12  5  2  6  3  5  6  5] [10 10  1  3  4  6  6  8 10 10  9  3  7  6  2  1  8 11  8  8]
accuracy: 0.25
9000 tensor(2.0514)
[ 9  6 10  1 11 12  6  4  3  5  6  4 11  1 11  5  5  8  6  3] [ 4 10 10  8 11 10 11  4  8  8  1  1 11  1  9 10  1  8  3

11600 tensor(2.1543)
[ 1  8  9  1  7  6  6  6  5  6  6  3  1  5  8 10  4  5  8  5] [ 9  1 10  1  7  8  6  6  7  1 11  8  1  3  1  8  6 10  8  7]
[12 10  2  5  5 12  8  7  4  5 11  6  1  4  1 10 11  1 10  5] [ 5  3  9  8  5 10 12  0 11  1  1  1  6  4 11  3  3  1 12 10]
accuracy: 0.30078125
11700 tensor(1.9408)
[ 5 11  8 10  4  1  9  1 11  1  5  1  6  1  3 11  5 11  8 11] [ 5  6  8 12  3  1  6  1 11  1  5  8  8  4 10 11 11 11  8  3]
[12  3 10 10  1  3  2  3  6  6  4  4  9  1  5  1  3  8  9 10] [ 5 10  3  1 10 10  7  6  6  1 11  1  6  2  5  7  8 10 10  3]
accuracy: 0.291015625
11800 tensor(2.0800)
[ 1  1  1  1  6  5  0  8  1  6  1  3  4 10  4  1  6 12  1  1] [11  3  6  6  6  6 10  3  6 10  1  3 11 10  4  6  1  5  8  6]
[ 9  5 11  3 11  6  5  3  6  3  4 10  5  5  6 10  4  5  6  9] [ 5  1 11  3 11  1 10 11  1 10 11  5  5 10 12 10  9  8  6  1]
accuracy: 0.337890625
11900 tensor(2.0020)
[10  6  5  8  6  4  3  8  4  3  5  4  5 11  3  4  9  2  4  1] [12 11  7  8  9  9  8  8  6  3  5  9  5  1  8

In [38]:
    accuracy = 0.0
    c = 0
    for j in range(100):
        for i in data_loader:
            c+=1
            print(c)
            p, m, l, mask, marr = i['p'], i['m'], i['l'], i['mask'], i['marr']
            tag_scores = model(p, m, mask)
            pred = tag_scores[0].max(1)[1].numpy()
            truth = torch.t(l)[0].numpy()
            print(pred[:20], truth[:20])
            accuracy += np.average(pred == truth)
            if (c>=1): break
    print("accuracy:", accuracy/c)

1
[ 3 10 10 11  9  4  1  1 10  8  3  5  8 10 11 10 10  4  3 10] [ 1  8  6 11  4  4  1  1  3  8  4  6  8  4  1  0  3  9  1  3]
2
[10  3 10  3  3 10 10  1  8  1  6  3 12  5  1 10  3  3  1  8] [10  5  5  3  4 10 12  6  1  8  8  3  3 10  8  3  6  7  6  1]
3
[ 5  1  1  7  5  4 10  1  9  1  8  3  8 10 11 10 10  8 11  5] [ 2  8  8  2 10  8 12  6  2  8  3 10 11  7 11  1  3  9  4  3]
4
[10  1 10  4  3  5  3  4  8  3 10  5  9  9 10 10  7  1  1  9] [ 1  1  6  1 10  2  8  4 12  3  8 10  9  4  8  8  8  5 10  1]
5
[ 7  3  1  6  1 10  5  4 11  5  6 10  4  1 10 10  3  8  5  9] [ 5  3  1  1  1 10  5  6 11  6  2  8 11  5  8 10  3  3  8  5]
6
[ 9  1  1  1  5  3  3  3  4  1  1 10  4  5 11  3 11  1  3 11] [ 4 11  8  4 12  3  3  3  4  8  9  8 11  4  6  6  9  9  8  6]
7
[12  8  5  5 10  6 10  6 10 10  9  1  3  1  9  8 10 10  2 10] [12  8  9  7 10 11  2 11  5  3  6 10  6  6 11  1  3  7  2 10]
8
[ 5  3  9  5  1 11  3  3  3  4  4  1  1  9 11  5 10  1  5  3] [10 10 12  3  8 11 10 12  8  4  4 11  1  2  1  6  6  6

[ 5  4  3  8  3 10 10  8  3  4 10 10 11  6  4  1  6 10 12  5] [ 2  6  3  8  3  8  6  6  1  4  1  8  6  6  4  5  1  8  2 10]
69
[ 5  1 12  1  1 11  8  5 10  6  8  1  2  6 11  8  3  5  3  9] [10  1  2  4  6  4  8  1  5 10  1 11  8  6  4  9 10  8  3  6]
70
[ 5  3  7 10  1  1  5 10 10  4  1  1  4  3 10  8  9 10  5 10] [ 8  9 11  0  6 12  9  3 10  4  6  8  6  8  5 10  4  8  5  3]
71
[ 8  3  1 10  3 10 10  5  1  1  1  3  6  3  1  8  6  1  1  6] [ 1  8  6  8  3 10  8 11  1  1  1  8  7  8 11  8 10 11  6  6]
72
[ 5  1  6  1  3 10 10  1  0  6  8  7  1  1  3 10  4  8  3  3] [ 5  6 11  8  8  1  3  1  4  8  5  7 11  6 10  5  1  6  8 10]
73
[10  6 10  1 10 10  5  1  8  1  9  1 11  1  5 12  3  8  8  4] [ 1  8 10  1  3 10 11 11  1  6  8 10 11  1  9  2  3  3 10  1]
74
[ 9  4 10  3  5  1 10  4  5  3  2  8 11  6  5  3  3 10  8  1] [ 9  6  5  8  7  6  5  6 10  3  3  8  6  6  5  8  5 12  8  8]
75
[ 4 11  3  1  1  3  3  8  1 10  9  0  3  3 11  1  6  5  4  6] [ 9 11 12  6  6  8 11 11 10  8  8  4 12  3  8  6 

Create the model:



Train the model:

